# Ejercicio

In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from pickle import dump


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv')
pd.set_option('display.max_columns', None)
df.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


0 comentario negativo
1 comentario positivo

Solo nos interesa la columna de los comentario, la del nombre de la app nos es indiferente por lo que la eliminamos.

In [3]:
df.drop(["package_name"], axis = 1, inplace = True)

Eliminar espacios y poner todo en minúscula.

In [5]:
df["review"] = df["review"].str.strip().str.lower()

División de los datos

In [13]:
X = df["review"]
y = df["polarity"]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
vec_model = CountVectorizer(stop_words = "english") # esto hace no contar ciertas palabras
X_train = vec_model.fit_transform(X_train).toarray()
X_test = vec_model.transform(X_test).toarray()

In [16]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Como la variable a precedir es binaria, hay dos posibles opciones, pero la predictora es categorica debemos usar el multinomial

Aplicamos el modelo

In [18]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

Vemos la predicción

In [19]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0])

In [22]:
precision_multi = accuracy_score(y_test, y_pred)
print(f"La precisión aplicanco el multinomial es de: {precision_multi}")

La precisión aplicanco el multinomial es de: 0.8156424581005587


Probamos con los demás

In [23]:
model_bern = BernoulliNB()
model_bern.fit(X_train, y_train)

BernoulliNB()

In [27]:
y_pred_bern = model_bern.predict(X_test)
precision_bern = accuracy_score(y_test, y_pred_bern)
print(f"La precisión aplicando Bernouilli es de: {precision_bern}")

La precisión aplicando Bernouilli es de: 0.770949720670391


In [28]:
model_gauss = GaussianNB()
model_gauss.fit(X_train, y_train)

GaussianNB()

In [29]:
y_pred_gauss = model_gauss.predict(X_test)
precision_gauss = accuracy_score(y_test, y_pred_gauss)
print(f"La precisión aplicando Gaussiano es de: {precision_gauss}")

La precisión aplicando Gaussiano es de: 0.8044692737430168


El mejor modelo para este caso es el elegido, el multinomial.

## Optimizar

El modelo multinomial tiene los hiperparámetros alpha y fit_prior que podemos ajustar

In [41]:
hyperparams = {
    "alpha": np.linspace(0.01, 10.0, 200), #poniendo esto tengo mucho mas rango que poniendo a mano posibles valores
    "fit_prior": [True, False]
}

grid = GridSearchCV(model, hyperparams, scoring = "accuracy", cv = 5)
grid

GridSearchCV(cv=5, estimator=MultinomialNB(),
             param_grid={'alpha': array([ 0.01      ,  0.06020101,  0.11040201,  0.16060302,  0.21080402,
        0.26100503,  0.31120603,  0.36140704,  0.41160804,  0.46180905,
        0.51201005,  0.56221106,  0.61241206,  0.66261307,  0.71281407,
        0.76301508,  0.81321608,  0.86341709,  0.91361809,  0.9638191 ,
        1.0140201 ,  1.06422111,  1.11442211,  1.16462312,  1.21482412,
        1.26502513,  1...
        8.54417085,  8.59437186,  8.64457286,  8.69477387,  8.74497487,
        8.79517588,  8.84537688,  8.89557789,  8.94577889,  8.9959799 ,
        9.0461809 ,  9.09638191,  9.14658291,  9.19678392,  9.24698492,
        9.29718593,  9.34738693,  9.39758794,  9.44778894,  9.49798995,
        9.54819095,  9.59839196,  9.64859296,  9.69879397,  9.74899497,
        9.79919598,  9.84939698,  9.89959799,  9.94979899, 10.        ]),
                         'fit_prior': [True, False]},
             scoring='accuracy')

In [42]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=MultinomialNB(),
             param_grid={'alpha': array([ 0.01      ,  0.06020101,  0.11040201,  0.16060302,  0.21080402,
        0.26100503,  0.31120603,  0.36140704,  0.41160804,  0.46180905,
        0.51201005,  0.56221106,  0.61241206,  0.66261307,  0.71281407,
        0.76301508,  0.81321608,  0.86341709,  0.91361809,  0.9638191 ,
        1.0140201 ,  1.06422111,  1.11442211,  1.16462312,  1.21482412,
        1.26502513,  1...
        8.54417085,  8.59437186,  8.64457286,  8.69477387,  8.74497487,
        8.79517588,  8.84537688,  8.89557789,  8.94577889,  8.9959799 ,
        9.0461809 ,  9.09638191,  9.14658291,  9.19678392,  9.24698492,
        9.29718593,  9.34738693,  9.39758794,  9.44778894,  9.49798995,
        9.54819095,  9.59839196,  9.64859296,  9.69879397,  9.74899497,
        9.79919598,  9.84939698,  9.89959799,  9.94979899, 10.        ]),
                         'fit_prior': [True, False]},
             scoring='accuracy')

In [45]:
mejor = grid.best_estimator_
print(f"Mejores parámetros: {mejor}")

Mejores parámetros: MultinomialNB(alpha=np.float64(1.8172361809045228), fit_prior=False)


In [46]:
model_opt = MultinomialNB(alpha = 1.8172361809045228, fit_prior = False)
model_opt.fit(X_train, y_train)
model_opt.fit(X_train, y_train)
y_pred_opt = model_opt.predict(X_test)
accuracy_score(y_test, y_pred_opt)

0.8156424581005587

Ha mejorado la precisión con la optimización.

## Guardamos el modelo optimizado

In [48]:
dump(model_opt, open("naive_bayes_reviews_multinomial_optimizado_42.sav", "wb"))